# Introduction
We are using a menu dataset from [AI Hub](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=242).
This dataset, originally intended for training vision models, includes a large number of labeled images and an organized Excel file (.xlsx) containing nutrient and category information.
However, we only need the menu labels and their corresponding nutrients from this organized file.

The dataset includes around 50,000 different kinds of detailed food data.
Among these, there are duplicates and some unusable entries, such as ingredients or overly detailed food items.
The following script has been created to refine and clean this food data.

> If you need the original dataset, refer this site. [AI hub](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=242)


## Refinement
### Fields we use
- Id
- Name
- Categories
- Carbohydrates
- Protein
- Fat
- Sugar
- Sodium (NaCl)
- Total energy

### Data folder structure
- root
    - datas
        - raw
            - raw_menu_nutrient.xlsx
        - refined
            - refined.xlsx

# imports

In [1]:
import pandas as pd
from pandas import DataFrame

# Load dataset

In [2]:
raw_datas: DataFrame = pd.read_excel('./datas/raw/raw_menu_nutrient.xlsx', engine='openpyxl', sheet_name=1)

# Extract DataFrame

In [3]:
raw_df: DataFrame = raw_datas.copy()

In [4]:
raw_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 225,Unnamed: 226,Unnamed: 227,Unnamed: 228,Unnamed: 229,Unnamed: 230,Unnamed: 231,Unnamed: 232,Unnamed: 233,Unnamed: 234
0,최종 DB 업데이트 일 : 2021-03-10,NaN,NaN,"● 이 파일은 통계용으로 사용하기 위한 것으로, 기존 ""-""값이 ""0""으로 변환된 ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NO,SAMPLE_ID,식품코드,DB군,상용제품,식품명,연도,지역 / 제조사,채취시기,식품대분류,...,냉산가용성물질(㎎),총 불포화지방산(g),식염상당량(g),회분(g),폐기율(%),가식부(%),산가용성물질(%),카페인(㎎),성분표출처,발행기관
3,1,D000006-94-AVG,D000006,음식,품목대표,꿩불고기,2019,충주,평균,구이류,...,0,0,0,5.8,0,0,0,0,식약처('16) 제4권,식품의약품안전처
4,2,D000007-ZZ-AVG,D000007,음식,품목대표,닭갈비,2019,전국(대표),평균,구이류,...,0,0,0,7,0,0,0,0,식약처('16) 제4권,식품의약품안전처


In [5]:
raw_df.columns = raw_df.iloc[2, :]
raw_df = raw_df.iloc[3:, :]
raw_df.set_index('NO', inplace=True)

# Extract features

In [6]:
df: DataFrame = raw_df.iloc[:, [1, 2, 3, 4, 6, 8, 9, 14, 18, 19, 20, 21, 44]]
df.columns = ['code', 'type', 'prod', 'name', 'manuf', 'cate_big', 'cate_specific', 'kcal', 'prot', 'fat', 'carb',
              'sugar', 'nat']

In [7]:
df.head()

,code,type,prod,name,manuf,cate_big,cate_specific,kcal,prot,fat,carb,sugar,nat
NO,,,,,,,,,,,,,
1,D000006,음식,품목대표,꿩불고기,충주,구이류,육류구이,368.8,33.5,8.5,39.7,16.9,1264.31
2,D000007,음식,품목대표,닭갈비,전국(대표),구이류,육류구이,595.61,45.9,25.8,44.9,21.2,1535.83
3,D000008,음식,품목대표,닭갈비,춘천,구이류,육류구이,558.47,45.5,31.6,23.1,8.5,1016.94
4,D000009,음식,품목대표,닭꼬치,전국(대표),구이류,육류구이,176.723,11.562,8.565,13.348,3.152,286.911
5,D000010,음식,품목대표,더덕구이,전국(대표),구이류,채소류구이,184,3.1,5.2,31.1,11.6,743.37


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52940 entries, 1 to 52940
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   code           52940 non-null  object
 1   type           52940 non-null  object
 2   prod           52940 non-null  object
 3   name           52940 non-null  object
 4   manuf          52938 non-null  object
 5   cate_big       52940 non-null  object
 6   cate_specific  52940 non-null  object
 7   kcal           52940 non-null  object
 8   prot           52940 non-null  object
 9   fat            52940 non-null  object
 10  carb           52940 non-null  object
 11  sugar          52940 non-null  object
 12  nat            52940 non-null  object
dtypes: object(13)
memory usage: 5.7+ MB


In [9]:
print(df.iloc[:, 7:].describe(), end='\n\n')
print(df.iloc[:, 7:].min(), end='\n\n')
print(df.iloc[:, 7:].max(), end='\n\n')
# Referring the documentation, they said that they had indicated missing value as -99. but minimum values are all 0. so there isn't any missing value. And it's so good.

           kcal     prot      fat     carb    sugar      nat
count   52940.0  52940.0  52940.0  52940.0  52940.0  52940.0
unique   5823.0   2253.0   2148.0   3954.0   2662.0   7037.0
top         0.0      0.0      0.0      0.0      0.0      0.0
freq     1305.0   9928.0  17345.0  10784.0   9452.0   6778.0

kcal     0.0
prot     0.0
fat      0.0
carb     0.0
sugar    0.0
nat      0.0
dtype: float64

kcal     86395.0
prot       600.0
fat      26374.0
carb     20999.0
sugar    16200.0
nat      40626.0
dtype: float64


In [10]:
df.nunique()

code             52328
type                 4
prod                 3
name             44471
manuf             3952
cate_big           179
cate_specific      299
kcal              5823
prot              2253
fat               2148
carb              3954
sugar             2662
nat               7037
dtype: int64

# Extract specific categories

In [12]:
name_only = df.name.str.contains('\(').apply(lambda x: not x)

meals_df = df[name_only & df.cate_big.isin(
    ['구이류', '국 및 탕류', '볶음류', '기타', '튀김류', '회류', '찌개 및 전골류', '찜류', '면 및 만두류', '밥류',
     '죽 및 스프류'])]  # 전.적 및 부침류(파전)
breads_df = df[name_only & df.cate_big.isin(['즉석섭취식품', '빵류', '시리얼류', '신선편의식품'])]
# desserts_df = df[name_only & df.cate_big.isin(['아이스크림류', '음료 및 차류', '빵류', '곡류 및 서류'])]

# meals
meals_unique = meals_df[
    meals_df.name.duplicated(keep=False).apply(lambda x: not x) | meals_df.name.duplicated(keep=False) & (
                meals_df.type == '음식') & (meals_df.manuf == '전국(대표)')]  # remove duplicates
exclude_specif = ['쌀밥.잡곡밥류', '어패류구이', '채소류구이', '채소류볶음', '어패류볶음', '기타', '어패류찜', '채소류튀김', '어패류튀김', '기타 튀김류', '채소류찜', '기타 찜류',
           '만두류']  # 어패류볶음(주꾸미볶음, 낙지볶음), 기타(월남쌈), 어패류찜(해물찜, 코다리찜, 오징어순대), 어패류튀김(생선까스)
manuf = ['충주', '전국(대표)', '서울특별시 마포구', '광양', '고흥', '수원', '기장', '태안', '삼척', '장흥', '괴산', '대전', '포항', '속초', '강릉', '정선',
         '안성', '밀양', '거제', '영주', '산청', '부안', '영동', '대구광역시 중구', '광주', '안동', '영암', '순천']

meals = meals_unique[meals_unique.cate_specific.isin(exclude_specif).apply(lambda x: not x) & meals_unique.manuf.isin(manuf)]
meals = meals[meals.name.str.contains('삼각김밥').apply(lambda x: not x)]

# breads
breads_unique = breads_df[
    breads_df.name.duplicated(keep=False).apply(lambda x: not x) | breads_df.name.duplicated(keep=False)]  # remove duplicates
exclude_specif = ['앙금빵류', '기타 빵류', '크림빵류', '케이크류', '페이스트리류', '식빵류', '튀김빵류(도넛, 꽈배기 등)']
manuf = ['전국(대표)']

breads = breads_unique[
    breads_unique.cate_specific.isin(exclude_specif).apply(lambda x: not x) & breads_unique.manuf.isin(manuf)]

# concat
menus = pd.concat([meals, breads])

# add inappropriately excepted datas
menu_names = ['파전', '주꾸미볶음', '낙지볶음', '월남쌈', '해물찜', '코다리찜', '오징어순대', '생선까스']
excepted = meals_df[meals_df.name.isin(menu_names)]

menus = pd.concat([menus, excepted])

menus

,code,type,prod,name,manuf,cate_big,cate_specific,kcal,prot,fat,carb,sugar,nat
NO,,,,,,,,,,,,,
1,D000006,음식,품목대표,꿩불고기,충주,구이류,육류구이,368.8,33.5,8.5,39.7,16.9,1264.31
2,D000007,음식,품목대표,닭갈비,전국(대표),구이류,육류구이,595.61,45.9,25.8,44.9,21.2,1535.83
4,D000009,음식,품목대표,닭꼬치,전국(대표),구이류,육류구이,176.723,11.562,8.565,13.348,3.152,286.911
7,D000012,음식,품목대표,돼지갈비,서울특별시 마포구,구이류,육류구이,240.32,19.5,14.4,8.1,4.5,404.66
9,D000016,음식,품목대표,불고기,광양,구이류,육류구이,395.29,34.4,25.2,7.8,7.9,560.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,D000426,음식,품목대표,해물찜,전국(대표),찜류,어패류찜,400.08,42.6,9.8,35.4,5.5,2137.21
279,D000473,음식,품목대표,생선까스,전국(대표),튀김류,어패류튀김,653.441,24.334,41.096,46.56,1.958,788.623
6963,D018177,음식,품목대표,월남쌈,전국(대표),기타,기타,293,14.9,12.4,30.6,13.7,574.48


In [13]:
# check result if it includes "menu names or keywords"
menus[menus.name.astype(str).str.contains('라이스')]

,code,type,prod,name,manuf,cate_big,cate_specific,kcal,prot,fat,carb,sugar,nat
NO,,,,,,,,,,,,,
7001,D018215,음식,품목대표,오므라이스,전국(대표),밥류,볶음밥류,692,26,28.9,82.1,12.8,1730.11
7019,D018233,음식,품목대표,카레라이스,전국(대표),밥류,덮밥류,518,14.9,6.7,99.5,0.8,1007.05
7024,D018238,음식,품목대표,하이라이스,전국(대표),밥류,덮밥류,401,14.5,4.1,76.8,0,884.09


In [14]:
menus.to_excel('./datas/refined/refined.xlsx', index=False)